In [1]:
import numpy as np
from tqdm import trange
from astropy.io import ascii
from astropy.table import Table
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import warnings
from astroquery.gaia import Gaia
chrome_options = Options()
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless")

In [7]:
def search_gaia(gaia_id, RA=None, dec=None, star=None):
    coord = SkyCoord(ra=RA, dec=dec, unit=(u.hourangle, u.deg), frame='icrs')
    width = u.Quantity(0.1, u.deg)
    height = u.Quantity(0.1, u.deg)
    r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
    i = np.where(r['DESIGNATION'] == gaia_id)
    Gmag = float(r['phot_g_mean_mag'][i])
    BPmag = float(r['phot_bp_mean_mag'][i])
    RPmag = float(r['phot_rp_mean_mag'][i])
    pmRA_gaia = f"{float(r['pmra'][i]): .4f}"
    pmdec_gaia = f"{float(r['pmdec'][i]): .4f}"

    BP_RP = BPmag-RPmag
    Rmag = Gmag + 0.29676 - 0.64728*(BP_RP) + 0.10141*(BP_RP)**2
    Rmag = f'{Rmag: .4f}'
    if BP_RP > 0.2 and BP_RP < 4.5:
        return Rmag, pmRA_gaia, pmdec_gaia
    else:
        print('The r mag is: '+str(i))
        warnings.warn(f'This Bp-Rp range (BP-RP = {BP_RP}) may not be valid. ')
        return Rmag, pmRA_gaia, pmdec_gaia
    
    
    
def search_exofop(star):
    driver = webdriver.Chrome() # '/home/tehan/Downloads/chromedriver-linux64/chromedriver'
    driver.get('https://exofop.ipac.caltech.edu/tess/')
    search = driver.find_element(By.NAME, value='target')
    search.send_keys(star)
    search.send_keys(Keys.ENTER)
    time.sleep(2)
    
    # RA/dec
    position = driver.find_elements(By.TAG_NAME, 'li')
    gaia_id = None
    for j in range(len(position)):
        if position[j].text.split(' ')[0] == 'RA/Dec':
            RA = position[j].text.split(' ')[4]
            dec = position[j].text.split(' ')[5]
        elif position[j].text.split(' ')[0] == 'Star':
            for k in range(len(position[j].text.split(', '))):
                if position[j].text.split(', ')[k][:4] == 'Gaia':
                    gaia_id = position[j].text.split(', ')[k]
            
    
    # proper motion
    table = driver.find_element(By.CLASS_NAME, 'inline-table')
    tableRows = table.find_element(By.TAG_NAME, 'tr')
    pmRA = tableRows.text.split(' ')[1]
    pmdec = tableRows.text.split(' ')[4]
    
    # magnitudes
    Jmag = '0'
    Vmag = '0'	
    Rmag = '0'
    Tmag = '0'
    magnitudes = driver.find_element(By.ID, 'myGrid6')
    mags = np.array(magnitudes.text.split("\n"))
    try:
        Jmag = mags[np.where(mags== 'J')[0][0] + 1].split(' ')[0]
    except:
        pass
    try:
        Vmag = mags[np.where(mags== 'V')[0][0] + 1].split(' ')[0]
    except:
        pass
    try:
        Rmag = mags[np.where(mags== 'r')[0][0] + 1].split(' ')[0]
    except IndexError:
        try:
            Rmag, pmRA_gaia, pmdec_gaia = search_gaia(gaia_id, RA=RA, dec=dec)
        except ValueError:
            print(f'No Gaia ID for {star}.')
            pass
    try:
        Tmag = mags[np.where(mags== 'TESS')[0][0] + 1].split(' ')[0]
    except:
        pass
    driver.quit()
    
    # if pmRA == '0':
    #     pmRA = pmRA_gaia
    # if pmdec == '0':
    #     pmdec = pmdec_gaia
    
    return [star, RA, dec, pmRA, pmdec, Jmag, Vmag, Rmag, Tmag]

# element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "myDynamicElement")))

In [11]:
search_exofop('TOI-6330')

INFO: Query finished. [astroquery.utils.tap.core]


['TOI-6330',
 '01:28:40.32',
 '+53:41:09.88',
 '-35.6682',
 '-45.0748',
 '12.368',
 '16.33',
 ' 14.2710',
 '13.7853']

In [12]:
path = '/home/tehan/Documents/SURFSUP/'
data = ascii.read(f'{path}SURFSUP.csv', data_start=1)
object_list = []
for i in trange(len(data['col2'])):
    object_list.append(search_exofop(data['col2'][i]))

# change the wrong minus sign
for lists in object_list:
    for sub in lists:
        sub.replace('−', '-')
   
object_list = Table(np.array(object_list), names = ['Name', 'RA','Dec','pmra','pmdec','Jmag','Vmag','Rmag','Tmag'])
ascii.write(object_list, f'{path}object_list.csv', format='csv', overwrite=True)  
object_list

  1%|▏         | 1/73 [00:06<08:23,  6.99s/it]

INFO: Query finished. [astroquery.utils.tap.core]


  4%|▍         | 3/73 [00:19<07:36,  6.52s/it]

INFO: Query finished. [astroquery.utils.tap.core]


  5%|▌         | 4/73 [00:26<07:43,  6.71s/it]

...

INFO: Query finished. [astroquery.utils.tap.core]


 99%|█████████▊| 72/73 [07:25<00:05,  5.91s/it]

INFO: Query finished. [astroquery.utils.tap.core]
No Gaia ID for TIC 471012349.


100%|██████████| 73/73 [07:29<00:00,  6.16s/it]


AttributeError: 'list' object has no attribute 'replace'